In this notebook the data from 5 chunks is being loaded. The next steps involve mering 5 tables into one and filtering.<br/> First applied filter is score>=0 which task is to eliminate some errors in the data (rows with score=0, are implicit feedback).<br/> Second filter is time filter, we keep only records where timestamp > 2015-01-01.<br/>
Then we filter games and users with number of **ratings** not smaller that given threshold (500 and 10 recpectively). This is coded as iterative procedure, we remove users, we remove games, we remove users, we remove games, as so on, until convergence (moment when all criterias are met).<br/>
Lastly we save the data.

In [1]:
import pandas as pd
import numpy as np
from time import perf_counter
from tqdm import tqdm

In [2]:
# VARIABLES
min_rating = 0
min_date = "2015-00-00 00:00:00"
min_n_r_users = 10
min_n_r_items = 500
data_path = "/home/mmarzec12/data/"
file_names = ["ratings_chunk_1.csv", "ratings_chunk_2.csv", "ratings_chunk_3.csv",
              "ratings_chunk_4.csv", "ratings_chunk_5.csv"]

## Merging files into 1 file and filtering

In [3]:
cols = ['user_name', 'game_id', 'score', 'timestamp', 'stat_own',
       'stat_preordered', 'stat_wishlist', 'stat_fortrade', 'stat_wanttoplay',
       'stat_prevowned', 'stat_want', 'stat_wanttobuy']
res = pd.DataFrame(columns=cols)
start = perf_counter()
for i, file_name in enumerate(file_names):
    df = pd.read_csv(data_path+file_name)
    filtered = df.loc[(df.score >= min_rating) & (df.timestamp > min_date)].drop(["Unnamed: 0"], axis=1)
    res = res.append(filtered)
    end = perf_counter()
    elapsed = end - start
    print("Loading file {} and adding it to dataframe took {} minutes.".format(i+1, round(elapsed/60, 2)))
    start = perf_counter()
print("Final dataframe shape is {}".format(res.shape))
print("We have {} different users (with ratings>0).".format(len(res.user_name.unique())))
print("We have {} different games.".format(len(res.game_id.unique())))

Loading file 1 and adding it to dataframe took 0.22 minutes.
Loading file 2 and adding it to dataframe took 0.23 minutes.
Loading file 3 and adding it to dataframe took 0.24 minutes.
Loading file 4 and adding it to dataframe took 0.26 minutes.
Loading file 5 and adding it to dataframe took 0.21 minutes.
Final dataframe shape is (30477976, 12)
We have 419468 different users (with ratings>0).
We have 107330 different games.


In [4]:
res["game_id"] = res["game_id"].astype(int)
res = res.reset_index()

In [5]:
res.timestamp.max()

'2019-08-30 09:15:25'

In [5]:
res.head()

,index,user_name,game_id,score,timestamp,stat_own,stat_preordered,stat_wishlist,stat_fortrade,stat_wanttoplay,stat_prevowned,stat_want,stat_wanttobuy
0,1,hsjx945,1,8.0,2019-08-27 10:20:16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,RuffRyder,1,0.0,2019-07-01 16:38:47,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,4,Olmestig,1,8.0,2019-08-25 12:46:35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,grassquit,1,9.0,2017-07-01 08:35:49,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,erzengel,1,0.0,2019-06-30 15:08:39,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


## Removing users and items from tail

In [17]:
i = 1 # iterations counter
# initiate rating count for games
tmp = res.copy()
items_g = tmp[tmp.score>0].groupby("game_id")["score"].count().reset_index().rename({"score":"n_ratings"}, axis=1)


while items_g.n_ratings.min() < min_n_r_items:
    # filtering games
    filtered = items_g[items_g.n_ratings > min_n_r_items]
    tmp = tmp.merge(filtered, on="game_id", how="inner").drop(["n_ratings"], axis=1)
    diff_i = len(items_g) - len(filtered)
    
    # checking condition for users
    users_g = tmp[tmp.score>0].groupby("user_name")["score"].count().reset_index().rename({"score":"n_ratings"}, axis=1)
    if users_g.n_ratings.min() < min_n_r_users:
        # filtering users
        filtered = users_g[users_g.n_ratings > min_n_r_users]
        tmp = tmp.merge(filtered, on="user_name", how="inner").drop(["n_ratings"], axis=1)
        diff_u = len(users_g) - len(filtered)
    else:
        break
    
    print("After {} iteration {} users and {} games were removed.".format(i, diff_u, diff_i))
    
    # recalculate ratings for games
    items_g = tmp[tmp.score>0].groupby("game_id")["score"].count().reset_index().rename({"score":"n_ratings"}, axis=1)
    
    # increase iteration count
    i += 1

print("We have started with {} users and {} games".format(len(res.user_name.unique()), len(res.game_id.unique())))
print("We have finished with {} users and {} games".format(len(tmp.user_name.unique()), len(tmp.game_id.unique())))

After 1 iteration 110286 users and 66873 games were removed.
After 2 iteration 440 users and 103 games were removed.
We have started with 419468 users and 107330 games
We have finished with 109084 users and 2265 games


In [14]:
tmp.head(10)

,index,user_name,game_id,score,timestamp,stat_own,stat_preordered,stat_wishlist,stat_fortrade,stat_wanttoplay,stat_prevowned,stat_want,stat_wanttobuy
0,1,hsjx945,1,8.0,2019-08-27 10:20:16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16859,hsjx945,3,7.3,2017-07-02 06:56:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,69289,hsjx945,5,0.0,2017-03-12 04:54:54,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,108569,hsjx945,11,6.8,2017-03-12 05:20:14,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,166075,hsjx945,12,6.5,2017-06-02 05:56:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,512598,hsjx945,42,0.0,2017-12-26 10:31:59,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
6,1230269,hsjx945,172,6.0,2017-11-11 04:55:58,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2168372,hsjx945,478,7.0,2016-10-22 03:32:18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2731975,hsjx945,760,7.0,2017-03-12 05:20:28,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5251333,hsjx945,2651,8.0,2017-03-12 04:47:55,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [18]:
tmp.timestamp.min()

'2015-01-01 00:01:56'

In [19]:
tmp.shape

(10994754, 13)

In [20]:
#tmp.to_csv(data_path+"bgg_ratings_merged_filtered.csv")
tmp.to_csv(data_path+"bgg_ratings_merged_filtered_2015_10_500.csv")